Варіант №5 Розробити програму для обліку кімнат в гуртожитку. Кожна кімната
може бути двомісною або трьохмісною. Необхідно зберігати інформацію
про тип кімнати та всіх її мешканців: ПІБ, дата народження, факультет,
група, форма навчання(державне замовлення чи за кошти фізичних осіб).
Комендант гуртожитку повинен формувати звіт у вигляді текстового
файлу про кожну кімнату(тип кімнати, кількість мешканців, квартплата
для кожного мешканця та інформація про всіх мешканців). Поставлене
завдання реалізувати за допомогою шаблону проектування Прототип.

In [4]:
import copy

class Student:
    def __init__(self, full_name, birth_date, faculty, group, form_of_study):
        self.full_name = full_name
        self.birth_date = birth_date
        self.faculty = faculty
        self.group = group
        self.form_of_study = form_of_study  # 'Держзамовлення' або 'Фіз. особи'

    def __str__(self):
        return (f"ПІБ: {self.full_name}, Дата народження: {self.birth_date}, "
                f"Факультет: {self.faculty}, Група: {self.group}, "
                f"Форма навчання: {self.form_of_study}")

class Room:
    def __init__(self, room_type):
        self.room_type = room_type  # 'Двомісна' або 'Трьохмісна'
        self.residents = []

    def add_resident(self, student):
        if len(self.residents) < (2 if self.room_type == 'Двомісна' else 3):
            self.residents.append(student)
        else:
            raise ValueError("У кімнаті вже достатньо мешканців")

    def calculate_rent(self, student):
        if student.form_of_study == 'Держзамовлення':
            return 500  # фіксована ціна для студентів на держзамовленні
        else:
            return 800  # ціна для студентів, що навчаються за кошти фіз. осіб

    def generate_report(self):
        with open(f'room_report_{id(self)}.txt', 'w') as f:
            f.write(f"Тип кімнати: {self.room_type}\n")
            f.write(f"Кількість мешканців: {len(self.residents)}\n")
            f.write("Мешканці:\n")
            for student in self.residents:
                f.write(f"{student}\n")
                f.write(f"Квартплата: {self.calculate_rent(student)} грн\n")
            f.write("\n")

    def clone(self):
        return copy.deepcopy(self)

# Приклад використання
if __name__ == "__main__":
    # Створюємо студентів
    student1 = Student("Іваненко Іван Іванович", "01.01.2001", "ФІОТ", "КМ-01", "Держзамовлення")
    student2 = Student("Петренко Петро Петрович", "02.02.2002", "ФІОТ", "КМ-02", "Фіз. особи")
    student3 = Student("Сидоренко Сидір Сидорович", "03.03.2003", "ФІОТ", "КМ-03", "Держзамовлення")

    # Створюємо двомісну кімнату
    room1 = Room("Двомісна")
    room1.add_resident(student1)
    room1.add_resident(student2)

    # Генеруємо звіт по кімнаті
    room1.generate_report()

    # Створюємо трьохмісну кімнату на основі двомісної
    room2 = room1.clone()
    room2.room_type = "Трьохмісна"
    room2.add_resident(student3)

    # Генеруємо звіт по трьохмісній кімнаті
    room2.generate_report()


Варіант №4 Продемонструвати шаблон проектування Одинак з ранньою
ініціалізацією на наступному прикладі. Розробити програму, в якій існує
лічильник для підрахунку кількості проданих товарів трьох груп(продукти
харчування, ліки, одяг). Вартість продукту визначається випадковим
чином з надбавкою 5% на продукти харчування, 10% на ліки та 15% на
одяг. Окрім кількості проданих товарів об’єкт лічильника повинен

зберігати сумарну вартість проданих товарів та файл, в якому повинен
записуватись час продажу кожного товару і найменування групи товарів.

In [6]:
import random
import time
from datetime import datetime

class SalesCounter:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(SalesCounter, cls).__new__(cls)
            cls._instance.food_count = 0
            cls._instance.medicine_count = 0
            cls._instance.clothes_count = 0
            cls._instance.total_value = 0.0
            cls._instance.log_file = 'sales_log.txt'
            # Очищаємо файл при першому запуску
            with open(cls._instance.log_file, 'w') as file:
                file.write("Час продажу, Група товарів, Ціна\n")
        return cls._instance

    def sell_item(self, group: str, base_price: float):
        if group == 'Продукти харчування':
            price = base_price * 1.05  # надбавка 5%
            self.food_count += 1
        elif group == 'Ліки':
            price = base_price * 1.10  # надбавка 10%
            self.medicine_count += 1
        elif group == 'Одяг':
            price = base_price * 1.15  # надбавка 15%
            self.clothes_count += 1
        else:
            raise ValueError("Невідома група товарів")

        self.total_value += price
        self._log_sale(group, price)

    def _log_sale(self, group: str, price: float):
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(self.log_file, 'a') as file:
            file.write(f"{current_time}, {group}, {price:.2f}\n")

    def get_summary(self):
        return {
            'Продукти харчування': self.food_count,
            'Ліки': self.medicine_count,
            'Одяг': self.clothes_count,
            'Сумарна вартість': self.total_value
        }

# Демонстрація
if __name__ == "__main__":
    counter = SalesCounter()

    # Продаж випадкових товарів
    groups = ['Продукти харчування', 'Ліки', 'Одяг']
    for _ in range(10):  # продаємо 10 товарів
        group = random.choice(groups)
        base_price = random.uniform(10, 100)  # випадкова базова ціна товару
        counter.sell_item(group, base_price)
        time.sleep(1)  # імітація затримки між продажами

    # Виведення підсумків
    summary = counter.get_summary()
    print("Кількість проданих товарів по групах:")
    for group, count in summary.items():
        if group != 'Сумарна вартість':
            print(f"{group}: {count}")
    print(f"Сумарна вартість проданих товарів: {summary['Сумарна вартість']:.2f}")


Кількість проданих товарів по групах:
Продукти харчування: 3
Ліки: 3
Одяг: 4
Сумарна вартість проданих товарів: 586.81
